In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict, Counter
import re
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# ========================= CONFIGURATION =========================
DATASET_PATH = "edf/"  # UPDATE THIS PATH
# =================================================================

print("="*80)
print("TUH TUSZ SEIZURE CORPUS - COMPREHENSIVE DATASET ANALYSIS")
print("="*80)

# Initialize data structures
dataset_stats = {
    'total_files': 0,
    'edf_files': [],
    'csv_files': [],
    'csv_bi_files': [],
    'patients': {},  # patient_id -> {sessions, recordings, demographics}
    'sessions': defaultdict(list),
    'file_sizes': defaultdict(list),
    'directory_depth': [],
    'train_eval_split': {'train': 0, 'eval': 0, 'dev': 0},
}

# Seizure-specific data
seizure_data = {
    'channel_annotations': defaultdict(list),  # per-channel annotations
    'seizure_events': [],  # bi-channel seizure events (TERM)
    'seizure_durations': [],
    'seizure_types': [],
    'background_durations': [],
    'channels_involved': defaultdict(int),
    'recording_durations': [],
}

# ==================== DATASET STRUCTURE EXPLORATION ====================
print("\n[1/7] Scanning dataset structure...")

for root, dirs, files in os.walk(DATASET_PATH):
    depth = root.replace(DATASET_PATH, '').count(os.sep)
    dataset_stats['directory_depth'].append(depth)
   
    # Determine train/eval/dev split
    if 'train' in root.lower():
        dataset_stats['train_eval_split']['train'] += len([f for f in files if f.endswith('.edf')])
    elif 'eval' in root.lower():
        dataset_stats['train_eval_split']['eval'] += len([f for f in files if f.endswith('.edf')])
    elif 'dev' in root.lower():
        dataset_stats['train_eval_split']['dev'] += len([f for f in files if f.endswith('.edf')])
   
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
       
        dataset_stats['total_files'] += 1
       
        # Extract patient and session info from path
        path_parts = Path(file_path).parts
        patient_id = None
        session_id = None
       
        # Find patient ID (format: aaaaaaaa)
        for part in path_parts:
            if re.match(r'^[a-z]{8}$', part):
                patient_id = part
            elif re.match(r'.*_s\d{3}', part):
                session_id = part
       
        if file.endswith('.edf'):
            dataset_stats['edf_files'].append(file_path)
            dataset_stats['file_sizes']['edf'].append(file_size)
           
            if patient_id:
                if patient_id not in dataset_stats['patients']:
                    dataset_stats['patients'][patient_id] = {
                        'sessions': set(),
                        'recordings': [],
                        'age': None,
                        'gender': None
                    }
                dataset_stats['patients'][patient_id]['recordings'].append(file_path)
                if session_id:
                    dataset_stats['patients'][patient_id]['sessions'].add(session_id)
               
        elif file.endswith('.csv_bi'):
            dataset_stats['csv_bi_files'].append(file_path)
            dataset_stats['file_sizes']['csv_bi'].append(file_size)
           
        elif file.endswith('.csv') and not file.endswith('.csv_bi'):
            dataset_stats['csv_files'].append(file_path)
            dataset_stats['file_sizes']['csv'].append(file_size)

print(f"✓ Scan complete: {dataset_stats['total_files']} files found")
print(f"  - Patients: {len(dataset_stats['patients'])}")
print(f"  - EDF files: {len(dataset_stats['edf_files'])}")
print(f"  - CSV files: {len(dataset_stats['csv_files'])}")
print(f"  - CSV_BI files: {len(dataset_stats['csv_bi_files'])}")

# ==================== PARSE CSV ANNOTATIONS ====================
print("\n[2/7] Parsing channel-level annotations (CSV files)...")

sample_size = min(200, len(dataset_stats['csv_files']))
sampled_csvs = np.random.choice(dataset_stats['csv_files'], sample_size, replace=False) if dataset_stats['csv_files'] else []

for csv_file in sampled_csvs:
    try:
        # Read CSV, skipping comment lines
        df = pd.read_csv(csv_file, comment='#')
       
        if not df.empty and 'label' in df.columns:
            # Extract recording duration from comments
            with open(csv_file, 'r') as f:
                for line in f:
                    if 'duration' in line:
                        match = re.search(r'duration = ([\d.]+)', line)
                        if match:
                            seizure_data['recording_durations'].append(float(match.group(1)))
                            break
           
            # Process annotations
            for _, row in df.iterrows():
                label = row['label'].strip() if pd.notna(row['label']) else 'unknown'
                channel = row['channel'] if pd.notna(row['channel']) else 'unknown'
                duration = row['stop_time'] - row['start_time'] if pd.notna(row['start_time']) and pd.notna(row['stop_time']) else 0
               
                seizure_data['channel_annotations'][label].append({
                    'channel': channel,
                    'duration': duration,
                    'start': row['start_time'] if pd.notna(row['start_time']) else 0,
                    'stop': row['stop_time'] if pd.notna(row['stop_time']) else 0
                })
               
                if label != 'bckg':  # Not background
                    seizure_data['seizure_types'].append(label)
                    seizure_data['channels_involved'][channel] += 1
                else:
                    seizure_data['background_durations'].append(duration)
                   
    except Exception as e:
        continue

print(f"✓ Parsed {sample_size} CSV files")
print(f"  - Unique seizure types: {len(set(seizure_data['seizure_types']))}")
print(f"  - Channels with seizure activity: {len(seizure_data['channels_involved'])}")

# ==================== PARSE CSV_BI (SEIZURE EVENTS) ====================
print("\n[3/7] Parsing seizure events (CSV_BI files)...")

sample_size_bi = min(200, len(dataset_stats['csv_bi_files']))
sampled_csv_bi = np.random.choice(dataset_stats['csv_bi_files'], sample_size_bi, replace=False) if dataset_stats['csv_bi_files'] else []

for csv_bi_file in sampled_csv_bi:
    try:
        df = pd.read_csv(csv_bi_file, comment='#')
       
        if not df.empty and 'label' in df.columns:
            for _, row in df.iterrows():
                if row['channel'] == 'TERM':  # Terminal event (overall seizure)
                    duration = row['stop_time'] - row['start_time']
                    seizure_data['seizure_events'].append({
                        'start': row['start_time'],
                        'stop': row['stop_time'],
                        'duration': duration,
                        'label': row['label'],
                        'file': os.path.basename(csv_bi_file)
                    })
                    seizure_data['seizure_durations'].append(duration)
                   
    except Exception as e:
        continue

print(f"✓ Parsed {sample_size_bi} CSV_BI files")
print(f"  - Total seizure events detected: {len(seizure_data['seizure_events'])}")

# ==================== PARSE EDF HEADERS (SAMPLE) ====================
print("\n[4/7] Analyzing EDF file metadata...")

edf_metadata = {
    'sampling_rates': [],
    'num_channels': [],
    'channel_names': [],
    'recording_durations_edf': [],
    'patient_ages': [],
    'patient_genders': []
}

sample_edf = min(50, len(dataset_stats['edf_files']))
sampled_edfs = np.random.choice(dataset_stats['edf_files'], sample_edf, replace=False) if dataset_stats['edf_files'] else []

for edf_file in sampled_edfs:
    try:
        with open(edf_file, 'rb') as f:
            header = f.read(256).decode('ascii', errors='ignore')
           
            # Extract patient info
            patient_info = header[8:88].strip()
            if 'Age:' in patient_info:
                age_match = re.search(r'Age:(\d+)', patient_info)
                if age_match:
                    edf_metadata['patient_ages'].append(int(age_match.group(1)))
           
            gender = header[8:9]
            if gender in ['M', 'F']:
                edf_metadata['patient_genders'].append(gender)
           
            # Number of signals
            num_signals = int(header[252:256].strip())
            edf_metadata['num_channels'].append(num_signals)
           
            # Read extended header for channel info
            extended_header_size = num_signals * 256
            extended = f.read(extended_header_size).decode('ascii', errors='ignore')
           
            # Extract channel names (16 chars each)
            for i in range(num_signals):
                start = i * 16
                end = start + 16
                if start < len(extended):
                    ch_name = extended[start:end].strip()
                    if ch_name and 'EEG' in ch_name:
                        edf_metadata['channel_names'].append(ch_name)
           
            # Extract sampling rate (last 8 chars of each signal's info)
            samples_per_record_start = num_signals * 216
            for i in range(min(3, num_signals)):  # Sample first 3 channels
                start = samples_per_record_start + i * 8
                end = start + 8
                if end <= len(extended):
                    try:
                        samp_rate = int(extended[start:end].strip())
                        if 50 <= samp_rate <= 1000:  # Reasonable EEG sampling rates
                            edf_metadata['sampling_rates'].append(samp_rate)
                    except:
                        pass
                       
    except Exception as e:
        continue

print(f"✓ Analyzed {sample_edf} EDF files")

# ==================== COMPUTE STATISTICS ====================
print("\n[5/7] Computing comprehensive statistics...")

stats_summary = {
    'Total Files': dataset_stats['total_files'],
    'EDF Files (EEG recordings)': len(dataset_stats['edf_files']),
    'CSV Files (channel annotations)': len(dataset_stats['csv_files']),
    'CSV_BI Files (seizure events)': len(dataset_stats['csv_bi_files']),
    'Unique Patients': len(dataset_stats['patients']),
    'Total Sessions': sum(len(p['sessions']) for p in dataset_stats['patients'].values()),
    'Recordings in Train': dataset_stats['train_eval_split']['train'],
    'Recordings in Eval': dataset_stats['train_eval_split']['eval'],
    'Recordings in Dev': dataset_stats['train_eval_split']['dev'],
}

# Patient statistics
recordings_per_patient = [len(p['recordings']) for p in dataset_stats['patients'].values()]
sessions_per_patient = [len(p['sessions']) for p in dataset_stats['patients'].values()]

# Seizure statistics
seizure_type_counts = Counter(seizure_data['seizure_types'])
channel_counts = Counter([ann['channel'] for anns in seizure_data['channel_annotations'].values() for ann in anns])

# ==================== VISUALIZATION ====================
print("\n[6/7] Generating comprehensive visualizations...")

fig = plt.figure(figsize=(24, 28))

# 1. Dataset Composition
ax1 = plt.subplot(5, 3, 1)
file_types = ['EDF\nRecordings', 'CSV\nChannel\nAnnotations', 'CSV_BI\nSeizure\nEvents']
file_counts = [len(dataset_stats['edf_files']), len(dataset_stats['csv_files']), len(dataset_stats['csv_bi_files'])]
colors = ['#3498db', '#2ecc71', '#e74c3c']
bars = ax1.bar(file_types, file_counts, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
ax1.set_title('Dataset File Composition', fontsize=14, fontweight='bold', pad=20)
ax1.grid(axis='y', alpha=0.3)
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 2. Train/Eval/Dev Split
ax2 = plt.subplot(5, 3, 2)
split_data = [v for k, v in dataset_stats['train_eval_split'].items() if v > 0]
split_labels = [k.upper() for k, v in dataset_stats['train_eval_split'].items() if v > 0]
if split_data:
    wedges, texts, autotexts = ax2.pie(split_data, labels=split_labels, autopct='%1.1f%%',
                                         colors=['#3498db', '#e74c3c', '#f39c12'], startangle=90)
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(12)
    ax2.set_title('Train/Eval/Dev Split', fontsize=14, fontweight='bold', pad=20)

# 3. Recordings per Patient
ax3 = plt.subplot(5, 3, 3)
ax3.hist(recordings_per_patient, bins=30, color='#9b59b6', alpha=0.7, edgecolor='black')
ax3.set_xlabel('Recordings per Patient', fontsize=12, fontweight='bold')
ax3.set_ylabel('Number of Patients', fontsize=12, fontweight='bold')
ax3.set_title(f'Patient Recording Distribution\n(μ={np.mean(recordings_per_patient):.1f}, σ={np.std(recordings_per_patient):.1f})',
              fontsize=14, fontweight='bold', pad=20)
ax3.axvline(np.mean(recordings_per_patient), color='red', linestyle='--', linewidth=2, label='Mean')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Seizure Type Distribution
ax4 = plt.subplot(5, 3, 4)
if seizure_type_counts:
    types = list(seizure_type_counts.keys())[:10]
    counts = [seizure_type_counts[t] for t in types]
    bars = ax4.barh(types, counts, color='#e67e22', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('Count', fontsize=12, fontweight='bold')
    ax4.set_title('Seizure Types (Top 10)', fontsize=14, fontweight='bold', pad=20)
    ax4.grid(axis='x', alpha=0.3)

# 5. Seizure Duration Distribution
ax5 = plt.subplot(5, 3, 5)
if seizure_data['seizure_durations']:
    ax5.hist(seizure_data['seizure_durations'], bins=40, color='#c0392b', alpha=0.7, edgecolor='black')
    ax5.set_xlabel('Duration (seconds)', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    mean_dur = np.mean(seizure_data['seizure_durations'])
    median_dur = np.median(seizure_data['seizure_durations'])
    ax5.set_title(f'Seizure Duration Distribution\n(μ={mean_dur:.1f}s, median={median_dur:.1f}s)',
                  fontsize=14, fontweight='bold', pad=20)
    ax5.axvline(mean_dur, color='yellow', linestyle='--', linewidth=2, label='Mean')
    ax5.axvline(median_dur, color='green', linestyle='--', linewidth=2, label='Median')
    ax5.legend()
    ax5.grid(alpha=0.3)

# 6. Recording Duration Distribution
ax6 = plt.subplot(5, 3, 6)
if seizure_data['recording_durations']:
    ax6.hist(seizure_data['recording_durations'], bins=40, color='#16a085', alpha=0.7, edgecolor='black')
    ax6.set_xlabel('Duration (seconds)', fontsize=12, fontweight='bold')
    ax6.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    mean_rec = np.mean(seizure_data['recording_durations'])
    ax6.set_title(f'Recording Duration Distribution\n(μ={mean_rec:.1f}s)',
                  fontsize=14, fontweight='bold', pad=20)
    ax6.axvline(mean_rec, color='red', linestyle='--', linewidth=2, label='Mean')
    ax6.legend()
    ax6.grid(alpha=0.3)

# 7. Most Active Channels (Seizure Activity)
ax7 = plt.subplot(5, 3, 7)
if seizure_data['channels_involved']:
    top_channels = sorted(seizure_data['channels_involved'].items(), key=lambda x: x[1], reverse=True)[:15]
    channels = [ch[0] for ch in top_channels]
    counts = [ch[1] for ch in top_channels]
    bars = ax7.barh(channels, counts, color='#8e44ad', alpha=0.7, edgecolor='black')
    ax7.set_xlabel('Seizure Event Count', fontsize=12, fontweight='bold')
    ax7.set_title('Top 15 Channels with Seizure Activity', fontsize=14, fontweight='bold', pad=20)
    ax7.grid(axis='x', alpha=0.3)

# 8. Sampling Rate Distribution
ax8 = plt.subplot(5, 3, 8)
if edf_metadata['sampling_rates']:
    sampling_rate_counts = Counter(edf_metadata['sampling_rates'])
    rates = list(sampling_rate_counts.keys())
    counts = list(sampling_rate_counts.values())
    ax8.bar(rates, counts, color='#27ae60', alpha=0.7, edgecolor='black', width=20)
    ax8.set_xlabel('Sampling Rate (Hz)', fontsize=12, fontweight='bold')
    ax8.set_ylabel('Number of Recordings', fontsize=12, fontweight='bold')
    ax8.set_title(f'Sampling Rate Distribution\n(Most common: {max(sampling_rate_counts, key=sampling_rate_counts.get)} Hz)',
                  fontsize=14, fontweight='bold', pad=20)
    ax8.grid(axis='y', alpha=0.3)

# 9. Number of Channels per Recording
ax9 = plt.subplot(5, 3, 9)
if edf_metadata['num_channels']:
    ax9.hist(edf_metadata['num_channels'], bins=20, color='#2980b9', alpha=0.7, edgecolor='black')
    ax9.set_xlabel('Number of Channels', fontsize=12, fontweight='bold')
    ax9.set_ylabel('Number of Recordings', fontsize=12, fontweight='bold')
    mean_ch = np.mean(edf_metadata['num_channels'])
    ax9.set_title(f'Channel Count Distribution\n(μ={mean_ch:.1f} channels)',
                  fontsize=14, fontweight='bold', pad=20)
    ax9.axvline(mean_ch, color='red', linestyle='--', linewidth=2, label='Mean')
    ax9.legend()
    ax9.grid(alpha=0.3)

# 10. Patient Age Distribution
ax10 = plt.subplot(5, 3, 10)
if edf_metadata['patient_ages']:
    ax10.hist(edf_metadata['patient_ages'], bins=30, color='#d35400', alpha=0.7, edgecolor='black')
    ax10.set_xlabel('Age (years)', fontsize=12, fontweight='bold')
    ax10.set_ylabel('Number of Patients', fontsize=12, fontweight='bold')
    mean_age = np.mean(edf_metadata['patient_ages'])
    ax10.set_title(f'Patient Age Distribution\n(μ={mean_age:.1f} years, range={min(edf_metadata["patient_ages"])}-{max(edf_metadata["patient_ages"])})',
                   fontsize=14, fontweight='bold', pad=20)
    ax10.axvline(mean_age, color='red', linestyle='--', linewidth=2, label='Mean')
    ax10.legend()
    ax10.grid(alpha=0.3)

# 11. Patient Gender Distribution
ax11 = plt.subplot(5, 3, 11)
if edf_metadata['patient_genders']:
    gender_counts = Counter(edf_metadata['patient_genders'])
    genders = list(gender_counts.keys())
    counts = list(gender_counts.values())
    colors_gender = ['#3498db' if g == 'M' else '#e91e63' for g in genders]
    bars = ax11.bar(genders, counts, color=colors_gender, alpha=0.7, edgecolor='black')
    ax11.set_ylabel('Count', fontsize=12, fontweight='bold')
    ax11.set_title('Patient Gender Distribution', fontsize=14, fontweight='bold', pad=20)
    ax11.grid(axis='y', alpha=0.3)
    for bar in bars:
        height = bar.get_height()
        ax11.text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 12. Sessions per Patient
ax12 = plt.subplot(5, 3, 12)
if sessions_per_patient:
    ax12.hist(sessions_per_patient, bins=20, color='#1abc9c', alpha=0.7, edgecolor='black')
    ax12.set_xlabel('Sessions per Patient', fontsize=12, fontweight='bold')
    ax12.set_ylabel('Number of Patients', fontsize=12, fontweight='bold')
    mean_sess = np.mean(sessions_per_patient)
    ax12.set_title(f'Patient Session Distribution\n(μ={mean_sess:.1f} sessions)',
                   fontsize=14, fontweight='bold', pad=20)
    ax12.axvline(mean_sess, color='red', linestyle='--', linewidth=2, label='Mean')
    ax12.legend()
    ax12.grid(alpha=0.3)

# 13. File Size Distribution by Type
ax13 = plt.subplot(5, 3, 13)
file_size_data = []
file_size_labels = []
for ftype, sizes in dataset_stats['file_sizes'].items():
    if sizes:
        file_size_data.extend(sizes)
        file_size_labels.extend([ftype.upper()] * len(sizes))

if file_size_data:
    import pandas as pd
    size_df = pd.DataFrame({'Size (MB)': file_size_data, 'Type': file_size_labels})
    sns.boxplot(data=size_df, x='Type', y='Size (MB)', ax=ax13, palette='Set2')
    ax13.set_title('File Size Distribution by Type', fontsize=14, fontweight='bold', pad=20)
    ax13.set_ylabel('Size (MB)', fontsize=12, fontweight='bold')
    ax13.grid(axis='y', alpha=0.3)

# 14. Seizure vs Background Time Ratio
ax14 = plt.subplot(5, 3, 14)
if seizure_data['seizure_durations'] and seizure_data['background_durations']:
    total_seizure = sum(seizure_data['seizure_durations'])
    total_background = sum(seizure_data['background_durations'][:len(seizure_data['seizure_durations'])])  # Match sample size
    data = [total_seizure, total_background]
    labels = [f'Seizure\n({total_seizure:.0f}s)', f'Background\n({total_background:.0f}s)']
    colors_pie = ['#e74c3c', '#95a5a6']
    wedges, texts, autotexts = ax14.pie(data, labels=labels, autopct='%1.1f%%',
                                         colors=colors_pie, startangle=90)
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(11)
    ax14.set_title('Seizure vs Background Time\n(Class Balance)', fontsize=14, fontweight='bold', pad=20)

# 15. Dataset Quality Metrics Table
ax15 = plt.subplot(5, 3, 15)
ax15.axis('tight')
ax15.axis('off')

quality_metrics = [
    ['Total Patients', f"{len(dataset_stats['patients']):,}"],
    ['Total Recordings', f"{len(dataset_stats['edf_files']):,}"],
    ['Avg Recordings/Patient', f"{np.mean(recordings_per_patient):.1f}"],
    ['Seizure Events Detected', f"{len(seizure_data['seizure_events']):,}"],
    ['Unique Seizure Types', f"{len(seizure_type_counts)}"],
    ['Avg Seizure Duration', f"{np.mean(seizure_data['seizure_durations']):.1f}s" if seizure_data['seizure_durations'] else "N/A"],
    ['Avg Recording Duration', f"{np.mean(seizure_data['recording_durations']):.1f}s" if seizure_data['recording_durations'] else "N/A"],
    ['Most Common Samp. Rate', f"{max(Counter(edf_metadata['sampling_rates']), key=Counter(edf_metadata['sampling_rates']).get)} Hz" if edf_metadata['sampling_rates'] else "N/A"],
]

table = ax15.table(cellText=quality_metrics, colWidths=[0.6, 0.4],
                  cellLoc='left', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)
for i in range(len(quality_metrics)):
    cell = table[(i, 0)]
    cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
    cell.set_text_props(weight='bold')
    cell = table[(i, 1)]
    cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
ax15.set_title('Key Dataset Metrics', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout(pad=3.0)
plt.savefig('viz_1_plots/tuh_tusz_comprehensive_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Visualizations saved to 'tuh_tusz_comprehensive_analysis.png'")

# ==================== DETAILED REPORT ====================
print("\n[7/7] Generating detailed report...")
print("\n" + "="*80)
print("DETAILED TUH TUSZ DATASET REPORT")
print("="*80)

print(f"\n📊 DATASET OVERVIEW:")
for key, value in stats_summary.items():
    print(f"  • {key}: {value:,}" if isinstance(value, int) else f"  • {key}: {value}")

print(f"\n🏥 PATIENT STATISTICS:")
print(f"  • Total unique patients: {len(dataset_stats['patients']):,}")
print(f"  • Total sessions: {stats_summary['Total Sessions']:,}")
print(f"  • Recordings per patient:")
print(f"    - Mean: {np.mean(recordings_per_patient):.2f} ± {np.std(recordings_per_patient):.2f}")
print(f"    - Median: {np.median(recordings_per_patient):.0f}")
print(f"    - Range: {min(recordings_per_patient)}-{max(recordings_per_patient)}")
print(f"  • Sessions per patient:")
print(f"    - Mean: {np.mean(sessions_per_patient):.2f} ± {np.std(sessions_per_patient):.2f}")
print(f"    - Median: {np.median(sessions_per_patient):.0f}")

if edf_metadata['patient_ages']:
    print(f"  • Patient age:")
    print(f"    - Mean: {np.mean(edf_metadata['patient_ages']):.1f} years")
    print(f"    - Range: {min(edf_metadata['patient_ages'])}-{max(edf_metadata['patient_ages'])} years")

if edf_metadata['patient_genders']:
    gender_dist = Counter(edf_metadata['patient_genders'])
    print(f"  • Gender distribution:")
    for gender, count in gender_dist.items():
        print(f"    - {gender}: {count} ({count/len(edf_metadata['patient_genders'])*100:.1f}%)")

print(f"\n🧠 SEIZURE ANNOTATION STATISTICS:")
print(f"  • Total seizure events (from CSV_BI): {len(seizure_data['seizure_events']):,}")
print(f"  • Unique seizure types: {len(seizure_type_counts)}")
print(f"  • Most common seizure types:")
if seizure_type_counts:
    for stype, count in seizure_type_counts.most_common(5):
        print(f"    - {stype}: {count} ({count/sum(seizure_type_counts.values())*100:.1f}%)")

if seizure_data['seizure_durations']:
    print(f"  • Seizure duration statistics:")
    print(f"    - Mean: {np.mean(seizure_data['seizure_durations']):.2f} seconds")
    print(f"    - Median: {np.median(seizure_data['seizure_durations']):.2f} seconds")
    print(f"    - Std: {np.std(seizure_data['seizure_durations']):.2f} seconds")
    print(f"    - Range: {min(seizure_data['seizure_durations']):.2f}-{max(seizure_data['seizure_durations']):.2f} seconds")
    print(f"    - Percentiles:")
    print(f"      * 25th: {np.percentile(seizure_data['seizure_durations'], 25):.2f}s")
    print(f"      * 75th: {np.percentile(seizure_data['seizure_durations'], 75):.2f}s")
    print(f"      * 90th: {np.percentile(seizure_data['seizure_durations'], 90):.2f}s")

print(f"\n📝 CHANNEL ANNOTATION ANALYSIS:")
print(f"  • Total channels with annotations: {len(seizure_data['channels_involved'])}")
print(f"  • Most active channels (seizure activity):")
if seizure_data['channels_involved']:
    top_channels = sorted(seizure_data['channels_involved'].items(), key=lambda x: x[1], reverse=True)[:10]
    for channel, count in top_channels:
        print(f"    - {channel}: {count} events")

print(f"\n⚖️ CLASS BALANCE ANALYSIS:")
if seizure_data['seizure_durations'] and seizure_data['background_durations']:
    total_seizure = sum(seizure_data['seizure_durations'])
    total_background = sum(seizure_data['background_durations'][:len(seizure_data['seizure_durations'])])
    ratio = total_background / total_seizure if total_seizure > 0 else 0
    print(f"  • Total seizure time: {total_seizure:.2f} seconds")
    print(f"  • Total background time (sampled): {total_background:.2f} seconds")
    print(f"  • Background-to-Seizure ratio: {ratio:.2f}:1")
    print(f"  • Seizure percentage: {total_seizure/(total_seizure+total_background)*100:.2f}%")
    print(f"  ⚠️  WARNING: Highly imbalanced dataset - consider:")
    print(f"    - Weighted loss functions")
    print(f"    - Oversampling seizure events")
    print(f"    - Focal loss or similar techniques")

print(f"\n💾 FILE SIZE ANALYSIS:")
for ftype, sizes in dataset_stats['file_sizes'].items():
    if sizes:
        print(f"  • {ftype.upper()} files:")
        print(f"    - Mean: {np.mean(sizes):.2f} MB")
        print(f"    - Median: {np.median(sizes):.2f} MB")
        print(f"    - Total: {sum(sizes):.2f} MB")
        print(f"    - Range: {min(sizes):.2f}-{max(sizes):.2f} MB")

print(f"\n🎛️ RECORDING METADATA:")
if edf_metadata['sampling_rates']:
    rate_dist = Counter(edf_metadata['sampling_rates'])
    print(f"  • Sampling rates:")
    for rate, count in rate_dist.most_common():
        print(f"    - {rate} Hz: {count} recordings ({count/len(edf_metadata['sampling_rates'])*100:.1f}%)")

if edf_metadata['num_channels']:
    print(f"  • Channels per recording:")
    print(f"    - Mean: {np.mean(edf_metadata['num_channels']):.1f}")
    print(f"    - Median: {np.median(edf_metadata['num_channels']):.0f}")
    print(f"    - Range: {min(edf_metadata['num_channels'])}-{max(edf_metadata['num_channels'])}")

if seizure_data['recording_durations']:
    print(f"  • Recording durations:")
    print(f"    - Mean: {np.mean(seizure_data['recording_durations']):.2f} seconds ({np.mean(seizure_data['recording_durations'])/60:.2f} minutes)")
    print(f"    - Median: {np.median(seizure_data['recording_durations']):.2f} seconds")
    print(f"    - Range: {min(seizure_data['recording_durations']):.2f}-{max(seizure_data['recording_durations']):.2f} seconds")

if edf_metadata['channel_names']:
    print(f"\n📡 CHANNEL NAMING PATTERNS:")
    channel_name_patterns = Counter([ch.split('-')[0] if '-' in ch else ch for ch in edf_metadata['channel_names'][:100]])
    print(f"  • Common channel prefixes (sample):")
    for pattern, count in channel_name_patterns.most_common(10):
        print(f"    - {pattern}: {count}")

print(f"\n📂 DATASET STRUCTURE:")
print(f"  • Maximum directory depth: {max(dataset_stats['directory_depth'])}")
print(f"  • Average directory depth: {np.mean(dataset_stats['directory_depth']):.1f}")
print(f"  • Train/Eval/Dev split:")
print(f"    - Train: {dataset_stats['train_eval_split']['train']:,} recordings ({dataset_stats['train_eval_split']['train']/len(dataset_stats['edf_files'])*100:.1f}%)")
print(f"    - Eval: {dataset_stats['train_eval_split']['eval']:,} recordings ({dataset_stats['train_eval_split']['eval']/len(dataset_stats['edf_files'])*100:.1f}%)")
if dataset_stats['train_eval_split']['dev'] > 0:
    print(f"    - Dev: {dataset_stats['train_eval_split']['dev']:,} recordings ({dataset_stats['train_eval_split']['dev']/len(dataset_stats['edf_files'])*100:.1f}%)")

print(f"\n🎯 DATA QUALITY INDICATORS:")
annotation_coverage = len(dataset_stats['csv_files']) / len(dataset_stats['edf_files']) * 100 if dataset_stats['edf_files'] else 0
print(f"  • Annotation coverage: {annotation_coverage:.1f}% (CSV/EDF ratio)")
bi_annotation_coverage = len(dataset_stats['csv_bi_files']) / len(dataset_stats['edf_files']) * 100 if dataset_stats['edf_files'] else 0
print(f"  • Bi-annotation coverage: {bi_annotation_coverage:.1f}% (CSV_BI/EDF ratio)")
print(f"  • Patients with multiple sessions: {sum(1 for p in dataset_stats['patients'].values() if len(p['sessions']) > 1)}")
print(f"  • Patients with >5 recordings: {sum(1 for c in recordings_per_patient if c > 5)}")
print(f"  • Average confidence score: {np.mean([1.0]*len(seizure_data['seizure_events'])):.2f} (from parsed data)")

print("\n" + "="*80)
print("💡 RECOMMENDATIONS FOR SEIZURE PREDICTION PROJECT")
print("="*80)

print("\n🔬 DATA PREPROCESSING:")
print("  1. Standardize sampling rates (resample to most common rate)")
print("  2. Handle channel naming inconsistencies")
print("  3. Apply bandpass filtering (typically 0.5-70 Hz for EEG)")
print("  4. Remove artifacts (eye blinks, muscle activity, electrical noise)")
print("  5. Normalize signal amplitude across recordings")

print("\n🎯 MODELING STRATEGY:")
print("  1. Address severe class imbalance:")
print("     - Use weighted cross-entropy loss")
print("     - Consider SMOTE or other oversampling techniques")
print("     - Try focal loss for hard examples")
print("  2. Feature engineering:")
print("     - Extract time-frequency features (wavelets, STFT)")
print("     - Compute spectral power in different frequency bands")
print("     - Calculate connectivity metrics between channels")
print("  3. Consider patient-specific models or transfer learning")
print("  4. Use sliding window approach for prediction")

print("\n📊 DATA SPLIT RECOMMENDATIONS:")
print("  1. Patient-level split (avoid data leakage)")
print("  2. Stratify by seizure type if possible")
print("  3. Consider temporal validation (earlier sessions for training)")
print("  4. Suggested split: 70% train, 15% validation, 15% test")

print("\n🧠 PREDICTION HORIZONS:")
if seizure_data['seizure_durations']:
    median_duration = np.median(seizure_data['seizure_durations'])
    print(f"  • Median seizure duration: {median_duration:.1f}s")
    print(f"  • Suggested prediction horizons:")
    print(f"    - Short-term: 5-30 seconds before onset")
    print(f"    - Medium-term: 30-180 seconds before onset")
    print(f"    - Long-term: 3-10 minutes before onset")

print("\n📈 EVALUATION METRICS:")
print("  1. Sensitivity/Recall (critical for seizure detection)")
print("  2. False Positive Rate (per hour)")
print("  3. Area Under ROC Curve (AUC-ROC)")
print("  4. Precision-Recall curve (better for imbalanced data)")
print("  5. Time to detection (latency)")

print("\n🔧 TECHNICAL CONSIDERATIONS:")
print("  1. GPU memory constraints with long recordings")
print("  2. Real-time processing requirements")
print("  3. Multi-channel spatial relationships")
print("  4. Temporal dependencies (consider RNNs, LSTMs, or Transformers)")
print("  5. Interpretability for clinical validation")

print("\n📚 SUGGESTED ARCHITECTURES:")
print("  1. 1D CNN + LSTM for temporal modeling")
print("  2. Graph Neural Networks (channels as nodes)")
print("  3. Transformer-based models (EEGFormer, BIOTFormer)")
print("  4. ResNet/EfficientNet adapted for 1D signals")
print("  5. Ensemble methods combining multiple approaches")

print("\n✅ NEXT STEPS:")
print("  1. ✓ Dataset exploration complete")
print("  2. → Load and visualize raw EEG signals (MNE-Python)")
print("  3. → Implement preprocessing pipeline")
print("  4. → Extract features and create training dataset")
print("  5. → Build baseline model")
print("  6. → Iterate and optimize")

print("\n" + "="*80)
print("Analysis complete! Generated 15 comprehensive visualizations.")
print("Output saved to: tuh_tusz_comprehensive_analysis.png")
print("="*80)

TUH TUSZ SEIZURE CORPUS - COMPREHENSIVE DATASET ANALYSIS

[1/7] Scanning dataset structure...
✓ Scan complete: 22092 files found
  - Patients: 675
  - EDF files: 7364
  - CSV files: 7364
  - CSV_BI files: 7364

[2/7] Parsing channel-level annotations (CSV files)...
✓ Parsed 200 CSV files
  - Unique seizure types: 5
  - Channels with seizure activity: 22

[3/7] Parsing seizure events (CSV_BI files)...
✓ Parsed 200 CSV_BI files
  - Total seizure events detected: 288

[4/7] Analyzing EDF file metadata...
✓ Analyzed 50 EDF files

[5/7] Computing comprehensive statistics...

[6/7] Generating comprehensive visualizations...
✓ Visualizations saved to 'tuh_tusz_comprehensive_analysis.png'

[7/7] Generating detailed report...

DETAILED TUH TUSZ DATASET REPORT

📊 DATASET OVERVIEW:
  • Total Files: 22,092
  • EDF Files (EEG recordings): 7,364
  • CSV Files (channel annotations): 7,364
  • CSV_BI Files (seizure events): 7,364
  • Unique Patients: 675
  • Total Sessions: 7,364
  • Recordings in Tra

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict, Counter
import re
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# ========================= CONFIGURATION =========================
DATASET_PATH = "edf/"  # UPDATE THIS PATH
OUTPUT_DIR = "viz_1_plots/"
# =================================================================

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("TUH TUSZ SEIZURE CORPUS - COMPREHENSIVE DATASET ANALYSIS")
print("="*80)

# Initialize data structures
dataset_stats = {
    'total_files': 0,
    'edf_files': [],
    'csv_files': [],
    'csv_bi_files': [],
    'patients': {},  # patient_id -> {sessions, recordings, demographics}
    'sessions': defaultdict(list),
    'file_sizes': defaultdict(list),
    'directory_depth': [],
    'train_eval_split': {'train': 0, 'eval': 0, 'dev': 0},
}

# Seizure-specific data
seizure_data = {
    'channel_annotations': defaultdict(list),  # per-channel annotations
    'seizure_events': [],  # bi-channel seizure events (TERM)
    'seizure_durations': [],
    'seizure_types': [],
    'background_durations': [],
    'channels_involved': defaultdict(int),
    'recording_durations': [],
}

# ==================== DATASET STRUCTURE EXPLORATION ====================
print("\n[1/7] Scanning dataset structure...")

for root, dirs, files in os.walk(DATASET_PATH):
    depth = root.replace(DATASET_PATH, '').count(os.sep)
    dataset_stats['directory_depth'].append(depth)
   
    # Determine train/eval/dev split
    root_lower = root.lower()
    if 'train' in root_lower:
        dataset_stats['train_eval_split']['train'] += len([f for f in files if f.endswith('.edf')])
    elif 'eval' in root_lower:
        dataset_stats['train_eval_split']['eval'] += len([f for f in files if f.endswith('.edf')])
    elif 'dev' in root_lower:
        dataset_stats['train_eval_split']['dev'] += len([f for f in files if f.endswith('.edf')])
   
    for file in files:
        file_path = os.path.join(root, file)
        
        # Check if file exists and get size
        if not os.path.exists(file_path):
            continue
            
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
       
        dataset_stats['total_files'] += 1
       
        # Extract patient and session info from path
        path_parts = Path(file_path).parts
        patient_id = None
        session_id = None
       
        # Find patient ID (format: aaaaaaaa)
        for part in path_parts:
            if re.match(r'^[a-z]{8}$', part):
                patient_id = part
            elif re.match(r'.*_s\d{3}', part):
                session_id = part
       
        if file.endswith('.edf'):
            dataset_stats['edf_files'].append(file_path)
            dataset_stats['file_sizes']['edf'].append(file_size)
           
            if patient_id:
                if patient_id not in dataset_stats['patients']:
                    dataset_stats['patients'][patient_id] = {
                        'sessions': set(),
                        'recordings': [],
                        'age': None,
                        'gender': None
                    }
                dataset_stats['patients'][patient_id]['recordings'].append(file_path)
                if session_id:
                    dataset_stats['patients'][patient_id]['sessions'].add(session_id)
               
        elif file.endswith('.csv_bi'):
            dataset_stats['csv_bi_files'].append(file_path)
            dataset_stats['file_sizes']['csv_bi'].append(file_size)
           
        elif file.endswith('.csv') and not file.endswith('.csv_bi'):
            dataset_stats['csv_files'].append(file_path)
            dataset_stats['file_sizes']['csv'].append(file_size)

print(f"✓ Scan complete: {dataset_stats['total_files']} files found")
print(f"  - Patients: {len(dataset_stats['patients'])}")
print(f"  - EDF files: {len(dataset_stats['edf_files'])}")
print(f"  - CSV files: {len(dataset_stats['csv_files'])}")
print(f"  - CSV_BI files: {len(dataset_stats['csv_bi_files'])}")

# ==================== PARSE CSV ANNOTATIONS ====================
print("\n[2/7] Parsing channel-level annotations (CSV files)...")

sample_size = min(200, len(dataset_stats['csv_files']))
sampled_csvs = np.random.choice(dataset_stats['csv_files'], sample_size, replace=False) if len(dataset_stats['csv_files']) > 0 else []

for csv_file in sampled_csvs:
    try:
        # Read CSV, skipping comment lines
        df = pd.read_csv(csv_file, comment='#')
       
        if not df.empty and 'label' in df.columns:
            # Extract recording duration from comments
            with open(csv_file, 'r') as f:
                for line in f:
                    if 'duration' in line.lower():
                        match = re.search(r'duration\s*=\s*([\d.]+)', line, re.IGNORECASE)
                        if match:
                            seizure_data['recording_durations'].append(float(match.group(1)))
                            break
           
            # Process annotations
            for _, row in df.iterrows():
                label = row['label'].strip() if pd.notna(row['label']) else 'unknown'
                channel = str(row['channel']) if pd.notna(row['channel']) else 'unknown'
                
                # Calculate duration safely
                start_time = row['start_time'] if pd.notna(row['start_time']) else 0
                stop_time = row['stop_time'] if pd.notna(row['stop_time']) else 0
                duration = stop_time - start_time if stop_time >= start_time else 0
               
                seizure_data['channel_annotations'][label].append({
                    'channel': channel,
                    'duration': duration,
                    'start': start_time,
                    'stop': stop_time
                })
               
                if label != 'bckg':  # Not background
                    seizure_data['seizure_types'].append(label)
                    seizure_data['channels_involved'][channel] += 1
                else:
                    seizure_data['background_durations'].append(duration)
                   
    except Exception as e:
        continue

print(f"✓ Parsed {len(sampled_csvs)} CSV files")
print(f"  - Unique seizure types: {len(set(seizure_data['seizure_types']))}")
print(f"  - Channels with seizure activity: {len(seizure_data['channels_involved'])}")

# ==================== PARSE CSV_BI (SEIZURE EVENTS) ====================
print("\n[3/7] Parsing seizure events (CSV_BI files)...")

sample_size_bi = min(200, len(dataset_stats['csv_bi_files']))
sampled_csv_bi = np.random.choice(dataset_stats['csv_bi_files'], sample_size_bi, replace=False) if len(dataset_stats['csv_bi_files']) > 0 else []

for csv_bi_file in sampled_csv_bi:
    try:
        df = pd.read_csv(csv_bi_file, comment='#')
       
        if not df.empty and 'label' in df.columns and 'channel' in df.columns:
            for _, row in df.iterrows():
                if str(row['channel']).strip() == 'TERM':  # Terminal event (overall seizure)
                    start_time = row['start_time'] if pd.notna(row['start_time']) else 0
                    stop_time = row['stop_time'] if pd.notna(row['stop_time']) else 0
                    duration = stop_time - start_time if stop_time >= start_time else 0
                    
                    if duration > 0:  # Only add valid durations
                        seizure_data['seizure_events'].append({
                            'start': start_time,
                            'stop': stop_time,
                            'duration': duration,
                            'label': row['label'] if pd.notna(row['label']) else 'unknown',
                            'file': os.path.basename(csv_bi_file)
                        })
                        seizure_data['seizure_durations'].append(duration)
                   
    except Exception as e:
        continue

print(f"✓ Parsed {len(sampled_csv_bi)} CSV_BI files")
print(f"  - Total seizure events detected: {len(seizure_data['seizure_events'])}")

# ==================== PARSE EDF HEADERS (SAMPLE) ====================
print("\n[4/7] Analyzing EDF file metadata...")

edf_metadata = {
    'sampling_rates': [],
    'num_channels': [],
    'channel_names': [],
    'recording_durations_edf': [],
    'patient_ages': [],
    'patient_genders': []
}

sample_edf = min(50, len(dataset_stats['edf_files']))
sampled_edfs = np.random.choice(dataset_stats['edf_files'], sample_edf, replace=False) if len(dataset_stats['edf_files']) > 0 else []

for edf_file in sampled_edfs:
    try:
        with open(edf_file, 'rb') as f:
            header = f.read(256).decode('ascii', errors='ignore')
           
            # Extract patient info
            patient_info = header[8:88].strip()
            if 'Age:' in patient_info:
                age_match = re.search(r'Age:(\d+)', patient_info)
                if age_match:
                    edf_metadata['patient_ages'].append(int(age_match.group(1)))
           
            gender = header[8:9]
            if gender in ['M', 'F']:
                edf_metadata['patient_genders'].append(gender)
           
            # Number of signals
            num_signals_str = header[252:256].strip()
            if num_signals_str:
                num_signals = int(num_signals_str)
                edf_metadata['num_channels'].append(num_signals)
           
                # Read extended header for channel info
                extended_header_size = num_signals * 256
                extended = f.read(extended_header_size).decode('ascii', errors='ignore')
           
                # Extract channel names (16 chars each)
                for i in range(num_signals):
                    start = i * 16
                    end = start + 16
                    if start < len(extended):
                        ch_name = extended[start:end].strip()
                        if ch_name and 'EEG' in ch_name:
                            edf_metadata['channel_names'].append(ch_name)
           
                # Extract sampling rate (last 8 chars of each signal's info)
                samples_per_record_start = num_signals * 216
                for i in range(min(3, num_signals)):  # Sample first 3 channels
                    start = samples_per_record_start + i * 8
                    end = start + 8
                    if end <= len(extended):
                        try:
                            samp_rate_str = extended[start:end].strip()
                            if samp_rate_str:
                                samp_rate = int(samp_rate_str)
                                if 50 <= samp_rate <= 1000:  # Reasonable EEG sampling rates
                                    edf_metadata['sampling_rates'].append(samp_rate)
                        except:
                            pass
                       
    except Exception as e:
        continue

print(f"✓ Analyzed {len(sampled_edfs)} EDF files")

# ==================== COMPUTE STATISTICS ====================
print("\n[5/7] Computing comprehensive statistics...")

stats_summary = {
    'Total Files': dataset_stats['total_files'],
    'EDF Files (EEG recordings)': len(dataset_stats['edf_files']),
    'CSV Files (channel annotations)': len(dataset_stats['csv_files']),
    'CSV_BI Files (seizure events)': len(dataset_stats['csv_bi_files']),
    'Unique Patients': len(dataset_stats['patients']),
    'Total Sessions': sum(len(p['sessions']) for p in dataset_stats['patients'].values()),
    'Recordings in Train': dataset_stats['train_eval_split']['train'],
    'Recordings in Eval': dataset_stats['train_eval_split']['eval'],
    'Recordings in Dev': dataset_stats['train_eval_split']['dev'],
}

# Patient statistics
recordings_per_patient = [len(p['recordings']) for p in dataset_stats['patients'].values()] if dataset_stats['patients'] else [0]
sessions_per_patient = [len(p['sessions']) for p in dataset_stats['patients'].values()] if dataset_stats['patients'] else [0]

# Seizure statistics
seizure_type_counts = Counter(seizure_data['seizure_types'])
channel_counts = Counter([ann['channel'] for anns in seizure_data['channel_annotations'].values() for ann in anns])

# ==================== VISUALIZATION ====================
print("\n[6/7] Generating comprehensive visualizations...")

def save_plot(filename):
    """Helper function to save plots"""
    filepath = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  ✓ Saved: {filename}")

# 1. Dataset Composition
plt.figure(figsize=(10, 6))
file_types = ['EDF\nRecordings', 'CSV\nChannel\nAnnotations', 'CSV_BI\nSeizure\nEvents']
file_counts = [len(dataset_stats['edf_files']), len(dataset_stats['csv_files']), len(dataset_stats['csv_bi_files'])]
colors = ['#3498db', '#2ecc71', '#e74c3c']
bars = plt.bar(file_types, file_counts, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
plt.ylabel('Count', fontsize=12, fontweight='bold')
plt.title('Dataset File Composition', fontsize=14, fontweight='bold', pad=20)
plt.grid(axis='y', alpha=0.3)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom', fontweight='bold', fontsize=11)
save_plot('01_dataset_composition.png')

# 2. Train/Eval/Dev Split
if any(dataset_stats['train_eval_split'].values()):
    plt.figure(figsize=(8, 8))
    split_data = [v for k, v in dataset_stats['train_eval_split'].items() if v > 0]
    split_labels = [k.upper() for k, v in dataset_stats['train_eval_split'].items() if v > 0]
    if split_data:
        wedges, texts, autotexts = plt.pie(split_data, labels=split_labels, autopct='%1.1f%%',
                                             colors=['#3498db', '#e74c3c', '#f39c12'], startangle=90)
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
            autotext.set_fontsize(12)
        plt.title('Train/Eval/Dev Split', fontsize=14, fontweight='bold', pad=20)
        save_plot('02_train_eval_split.png')

# 3. Recordings per Patient
if len(recordings_per_patient) > 1:
    plt.figure(figsize=(10, 6))
    plt.hist(recordings_per_patient, bins=min(30, max(recordings_per_patient)), color='#9b59b6', alpha=0.7, edgecolor='black')
    plt.xlabel('Recordings per Patient', fontsize=12, fontweight='bold')
    plt.ylabel('Number of Patients', fontsize=12, fontweight='bold')
    plt.title(f'Patient Recording Distribution\n(μ={np.mean(recordings_per_patient):.1f}, σ={np.std(recordings_per_patient):.1f})',
              fontsize=14, fontweight='bold', pad=20)
    plt.axvline(np.mean(recordings_per_patient), color='red', linestyle='--', linewidth=2, label='Mean')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('03_recordings_per_patient.png')

# 4. Seizure Type Distribution
if seizure_type_counts:
    plt.figure(figsize=(10, 6))
    types = list(seizure_type_counts.keys())[:10]
    counts = [seizure_type_counts[t] for t in types]
    plt.barh(types, counts, color='#e67e22', alpha=0.7, edgecolor='black')
    plt.xlabel('Count', fontsize=12, fontweight='bold')
    plt.title('Seizure Types (Top 10)', fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', alpha=0.3)
    save_plot('04_seizure_types.png')

# 5. Seizure Duration Distribution
if seizure_data['seizure_durations']:
    plt.figure(figsize=(10, 6))
    plt.hist(seizure_data['seizure_durations'], bins=40, color='#c0392b', alpha=0.7, edgecolor='black')
    plt.xlabel('Duration (seconds)', fontsize=12, fontweight='bold')
    plt.ylabel('Frequency', fontsize=12, fontweight='bold')
    mean_dur = np.mean(seizure_data['seizure_durations'])
    median_dur = np.median(seizure_data['seizure_durations'])
    plt.title(f'Seizure Duration Distribution\n(μ={mean_dur:.1f}s, median={median_dur:.1f}s)',
              fontsize=14, fontweight='bold', pad=20)
    plt.axvline(mean_dur, color='yellow', linestyle='--', linewidth=2, label='Mean')
    plt.axvline(median_dur, color='green', linestyle='--', linewidth=2, label='Median')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('05_seizure_duration.png')

# 6. Recording Duration Distribution
if seizure_data['recording_durations']:
    plt.figure(figsize=(10, 6))
    plt.hist(seizure_data['recording_durations'], bins=40, color='#16a085', alpha=0.7, edgecolor='black')
    plt.xlabel('Duration (seconds)', fontsize=12, fontweight='bold')
    plt.ylabel('Frequency', fontsize=12, fontweight='bold')
    mean_rec = np.mean(seizure_data['recording_durations'])
    plt.title(f'Recording Duration Distribution\n(μ={mean_rec:.1f}s)',
              fontsize=14, fontweight='bold', pad=20)
    plt.axvline(mean_rec, color='red', linestyle='--', linewidth=2, label='Mean')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('06_recording_duration.png')

# 7. Most Active Channels (Seizure Activity)
if seizure_data['channels_involved']:
    plt.figure(figsize=(10, 8))
    top_channels = sorted(seizure_data['channels_involved'].items(), key=lambda x: x[1], reverse=True)[:15]
    channels = [ch[0] for ch in top_channels]
    counts = [ch[1] for ch in top_channels]
    plt.barh(channels, counts, color='#8e44ad', alpha=0.7, edgecolor='black')
    plt.xlabel('Seizure Event Count', fontsize=12, fontweight='bold')
    plt.title('Top 15 Channels with Seizure Activity', fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', alpha=0.3)
    save_plot('07_active_channels.png')

# 8. Sampling Rate Distribution
if edf_metadata['sampling_rates']:
    plt.figure(figsize=(10, 6))
    sampling_rate_counts = Counter(edf_metadata['sampling_rates'])
    rates = sorted(list(sampling_rate_counts.keys()))
    counts = [sampling_rate_counts[r] for r in rates]
    plt.bar(rates, counts, color='#27ae60', alpha=0.7, edgecolor='black', width=20)
    plt.xlabel('Sampling Rate (Hz)', fontsize=12, fontweight='bold')
    plt.ylabel('Number of Recordings', fontsize=12, fontweight='bold')
    plt.title(f'Sampling Rate Distribution\n(Most common: {max(sampling_rate_counts, key=sampling_rate_counts.get)} Hz)',
              fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='y', alpha=0.3)
    save_plot('08_sampling_rates.png')

# 9. Number of Channels per Recording
if edf_metadata['num_channels']:
    plt.figure(figsize=(10, 6))
    plt.hist(edf_metadata['num_channels'], bins=20, color='#2980b9', alpha=0.7, edgecolor='black')
    plt.xlabel('Number of Channels', fontsize=12, fontweight='bold')
    plt.ylabel('Number of Recordings', fontsize=12, fontweight='bold')
    mean_ch = np.mean(edf_metadata['num_channels'])
    plt.title(f'Channel Count Distribution\n(μ={mean_ch:.1f} channels)',
              fontsize=14, fontweight='bold', pad=20)
    plt.axvline(mean_ch, color='red', linestyle='--', linewidth=2, label='Mean')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('09_channel_count.png')

# 10. Patient Age Distribution
if edf_metadata['patient_ages']:
    plt.figure(figsize=(10, 6))
    plt.hist(edf_metadata['patient_ages'], bins=30, color='#d35400', alpha=0.7, edgecolor='black')
    plt.xlabel('Age (years)', fontsize=12, fontweight='bold')
    plt.ylabel('Number of Patients', fontsize=12, fontweight='bold')
    mean_age = np.mean(edf_metadata['patient_ages'])
    plt.title(f'Patient Age Distribution\n(μ={mean_age:.1f} years, range={min(edf_metadata["patient_ages"])}-{max(edf_metadata["patient_ages"])})',
               fontsize=14, fontweight='bold', pad=20)
    plt.axvline(mean_age, color='red', linestyle='--', linewidth=2, label='Mean')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('10_patient_age.png')

# 11. Patient Gender Distribution
if edf_metadata['patient_genders']:
    plt.figure(figsize=(8, 6))
    gender_counts = Counter(edf_metadata['patient_genders'])
    genders = list(gender_counts.keys())
    counts = list(gender_counts.values())
    colors_gender = ['#3498db' if g == 'M' else '#e91e63' for g in genders]
    bars = plt.bar(genders, counts, color=colors_gender, alpha=0.7, edgecolor='black')
    plt.ylabel('Count', fontsize=12, fontweight='bold')
    plt.title('Patient Gender Distribution', fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='y', alpha=0.3)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=11)
    save_plot('11_patient_gender.png')

# 12. Sessions per Patient
if len(sessions_per_patient) > 1:
    plt.figure(figsize=(10, 6))
    plt.hist(sessions_per_patient, bins=min(20, max(sessions_per_patient)), color='#1abc9c', alpha=0.7, edgecolor='black')
    plt.xlabel('Sessions per Patient', fontsize=12, fontweight='bold')
    plt.ylabel('Number of Patients', fontsize=12, fontweight='bold')
    mean_sess = np.mean(sessions_per_patient)
    plt.title(f'Patient Session Distribution\n(μ={mean_sess:.1f} sessions)',
               fontsize=14, fontweight='bold', pad=20)
    plt.axvline(mean_sess, color='red', linestyle='--', linewidth=2, label='Mean')
    plt.legend()
    plt.grid(alpha=0.3)
    save_plot('12_sessions_per_patient.png')

# 13. File Size Distribution by Type
file_size_data = []
file_size_labels = []
for ftype, sizes in dataset_stats['file_sizes'].items():
    if sizes:
        file_size_data.extend(sizes)
        file_size_labels.extend([ftype.upper()] * len(sizes))

if file_size_data:
    plt.figure(figsize=(10, 6))
    size_df = pd.DataFrame({'Size (MB)': file_size_data, 'Type': file_size_labels})
    sns.boxplot(data=size_df, x='Type', y='Size (MB)', palette='Set2')
    plt.title('File Size Distribution by Type', fontsize=14, fontweight='bold', pad=20)
    plt.ylabel('Size (MB)', fontsize=12, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    save_plot('13_file_sizes.png')

# 14. Seizure vs Background Time Ratio
if seizure_data['seizure_durations'] and seizure_data['background_durations']:
    plt.figure(figsize=(8, 8))
    total_seizure = sum(seizure_data['seizure_durations'])
    total_background = sum(seizure_data['background_durations'][:len(seizure_data['seizure_durations'])])  # Match sample size
    data = [total_seizure, total_background]
    labels = [f'Seizure\n({total_seizure:.0f}s)', f'Background\n({total_background:.0f}s)']
    colors_pie = ['#e74c3c', '#95a5a6']
    wedges, texts, autotexts = plt.pie(data, labels=labels, autopct='%1.1f%%',
                                         colors=colors_pie, startangle=90)
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(11)
    plt.title('Seizure vs Background Time\n(Class Balance)', fontsize=14, fontweight='bold', pad=20)
    save_plot('14_class_balance.png')

# 15. Dataset Quality Metrics Table
fig, ax = plt.subplots(figsize=(10, 8))
ax.axis('tight')
ax.axis('off')

quality_metrics = [
    ['Total Patients', f"{len(dataset_stats['patients']):,}"],
    ['Total Recordings', f"{len(dataset_stats['edf_files']):,}"],
    ['Avg Recordings/Patient', f"{np.mean(recordings_per_patient):.1f}"],
    ['Seizure Events Detected', f"{len(seizure_data['seizure_events']):,}"],
    ['Unique Seizure Types', f"{len(seizure_type_counts)}"],
    ['Avg Seizure Duration', f"{np.mean(seizure_data['seizure_durations']):.1f}s" if seizure_data['seizure_durations'] else "N/A"],
    ['Avg Recording Duration', f"{np.mean(seizure_data['recording_durations']):.1f}s" if seizure_data['recording_durations'] else "N/A"],
    ['Most Common Samp. Rate', f"{max(Counter(edf_metadata['sampling_rates']), key=Counter(edf_metadata['sampling_rates']).get)} Hz" if edf_metadata['sampling_rates'] else "N/A"],
]

table = ax.table(cellText=quality_metrics, colWidths=[0.6, 0.4],
                  cellLoc='left', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)
for i in range(len(quality_metrics)):
    cell = table[(i, 0)]
    cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
    cell.set_text_props(weight='bold')
    cell = table[(i, 1)]
    cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
ax.set_title('Key Dataset Metrics', fontsize=14, fontweight='bold', pad=20)
save_plot('15_summary_metrics.png')

print(f"\n✓ All visualizations saved to '{OUTPUT_DIR}'")

TUH TUSZ SEIZURE CORPUS - COMPREHENSIVE DATASET ANALYSIS

[1/7] Scanning dataset structure...
✓ Scan complete: 22092 files found
  - Patients: 675
  - EDF files: 7364
  - CSV files: 7364
  - CSV_BI files: 7364

[2/7] Parsing channel-level annotations (CSV files)...
✓ Parsed 200 CSV files
  - Unique seizure types: 4
  - Channels with seizure activity: 22

[3/7] Parsing seizure events (CSV_BI files)...
✓ Parsed 200 CSV_BI files
  - Total seizure events detected: 348

[4/7] Analyzing EDF file metadata...
✓ Analyzed 50 EDF files

[5/7] Computing comprehensive statistics...

[6/7] Generating comprehensive visualizations...
  ✓ Saved: 01_dataset_composition.png
  ✓ Saved: 02_train_eval_split.png
  ✓ Saved: 03_recordings_per_patient.png
  ✓ Saved: 04_seizure_types.png
  ✓ Saved: 05_seizure_duration.png
  ✓ Saved: 06_recording_duration.png
  ✓ Saved: 07_active_channels.png
  ✓ Saved: 08_sampling_rates.png
  ✓ Saved: 09_channel_count.png
  ✓ Saved: 10_patient_age.png
  ✓ Saved: 12_sessions_per_